In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Hello and Welcome to the start of Ayan's Attempt to Geofferey Hinton's Fellowship

I am honored to participate in such an event and present my work.

Let's begin with the notebook then.

# Contents
## 1. [Libraries](#libraries)
## 2. [Data Visualization](#data_visualization)
## 3. [Feature Engineering](#feature_engineering)
## 4. [Data Modelling](#data_modelling)
## 5. [Output](#output)


# Libraries<a id='libraries'></a>
Let's start with all the imports and get done with the boring part

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)

import re
from category_encoders import LeaveOneOutEncoder
from sklearn.preprocessing import OrdinalEncoder # Not Needed Right now
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import optuna
import lightgbm
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Data Visualization<a id='data_visualization'></a>
## Let's look with what data we will be working with in this Problem

In [ ]:
train=pd.read_csv('/kaggle/input/ghf1-hackathon/Training Data.csv')
test=pd.read_csv('/kaggle/input/ghf1-hackathon/Test Data.csv')
sub=pd.read_csv('/kaggle/input/ghf1-hackathon/Sample Prediction Dataset.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.shape

## So the data has 2.5 lakh entires, that's a lot of credit card data

## Let's see what is the distribution of the target column in this data

In [ ]:
train['risk_flag'].value_counts()/len(train)*100

## Looks like the data is highly skewed.

## Check for null values in train and test set

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

## So no null values, the next step is to go for getting info from train set

## As see if we can find anything solid

In [ ]:
train.nunique()

# Id column looks a bit redundant here, dropping it will be the best thing to do

In [ ]:
total_columns=['age', 'experience', 'married', 'house_ownership','car_ownership', 'profession', 'city',
               'state', 'current_job_years','current_house_years',]

for i in total_columns:
    print(f'Column :{i} New Elements :{set(test[i].unique())-set(train[i].unique())}')
    print(f'Number of New Elements {len(set(test[i].unique())-set(train[i].unique()))}')
    print('')

### This is interesting, we can see that we have a lot of entries with '-' , 'upper case' , '_' or anything else creating a new entry

### Before we proceed any further, let's fix this error

In [ ]:
def convert_profession(x):
    prof = x
    prof = re.sub(r'\-',' ',prof)
    prof = re.sub(r'\_',' ',prof)
    prof = re.sub(r'\[[0-9]\]',' ',prof)
    prof = prof.split()
    prof = ' '.join(prof)
    
    prof = prof.lower()
    return prof

def convert_state(x):
    state = x
    state = re.sub(r'\-',' ',state)
    state = re.sub(r'\_',' ',state)
    state = re.sub(r'\[[0-9]\]',' ',state)
    state = state.split()
    state = ' '.join(state)
    
    state = state.lower()
    return state

def convert_city(x):
    city = x
    city = re.sub(r'\-',' ',city)
    city = re.sub(r'\_',' ',city)
    city = re.sub(r'\[[0-9][0-9]*\]',' ',city)
    city = city.split()
    city = ' '.join(city)
    
    city = city.lower()
    return city

train['profession'] = train['profession'].apply(convert_profession)
test['profession'] = test['profession'].apply(convert_profession)

train['state'] = train['state'].apply(convert_state)
test['state'] = test['state'].apply(convert_state)

train['city'] = train['city'].apply(convert_city)
test['city'] = test['city'].apply(convert_city)

### Confirm it for surety, sice being extra carefull is never bad 

In [ ]:
total_columns=['age', 'experience', 'married', 'house_ownership','car_ownership', 'profession', 'city',
               'state', 'current_job_years','current_house_years',]

for i in total_columns:
    print(f'Column :{i} New Elements :{set(test[i].unique())-set(train[i].unique())}')
    print(f'Number of New Elements {len(set(test[i].unique())-set(train[i].unique()))}')
    print('')

### Done and Done

### Moving on now, some basic visualisations for inferences 

In [ ]:
sns.distplot(train['income'],bins=1000,norm_hist=False,kde=True)

In [ ]:
sns.distplot(train['age'],kde=False)

In [ ]:
sns.boxplot(x=train['experience'],y=train['age'])

In [ ]:
sns.barplot(x='experience',y='age',data=train)

In [ ]:
sns.barplot(x='experience',y='income',hue='married',data=train)

### Looks like married people are paid less as they gather experience 

### TAKE THAT SOCIETY AND SOCIAL NORMS!

In [ ]:
sns.barplot(x='house_ownership',y='income',data=train)

In [ ]:
train.groupby('experience')['age'].median()

### Now, let's be honest with ourselves 

### This looks very beautiful but not a lot can actually be inferred from it, so let's jump right into feature engineering 

## Feature Engineering <a id='feature_engineering'></a>

### Time to get our hands dirty and play with the data 

### First things first let's check for duplicate entries

In [ ]:
train.drop(columns=['Id'],inplace=True)
test.drop(columns=['id'],inplace=True)

## For this question, as my model was becoming biased I made synthetic data using the available columns

In [ ]:
cat_col = ['married','house_ownership','car_ownership','profession','city','state']

for col in cat_col:
    for col1 in cat_col:
        if col != col1:
            train[col + '_+_'+col1] = train[col] + train[col1]
            test[col + '_+_'+col1] = test[col] + test[col1]

## Ended up dropping Profession, State and City because of their high VIF factor and dropping them gave me a better result

In [ ]:
train.drop(columns=['profession','state','city'],inplace=True)
test.drop(columns=['profession','state','city'],inplace=True)

In [ ]:
new_col=['house_ownership','married','car_ownership',
       'married_+_house_ownership', 'married_+_car_ownership','married_+_profession', 'married_+_city',
    'married_+_state','house_ownership_+_car_ownership','house_ownership_+_profession',
    'house_ownership_+_city','house_ownership_+_state', 'car_ownership_+_profession','car_ownership_+_city',
    'car_ownership_+_state', 'profession_+_city', 'profession_+_state','city_+_state', 
        ]
drop_cols_cat=['house_ownership_+_married','car_ownership_+_married','profession_+_married','city_+_married',
           'state_+_married','car_ownership_+_house_ownership','profession_+_house_ownership',
           'city_+_house_ownership','state_+_house_ownership','profession_+_car_ownership',
           'city_+_car_ownership','state_+_car_ownership','city_+_profession','state_+_profession',
           'state_+_city']

## Removing the Duplicate entries

In [ ]:
train.drop(columns=drop_cols_cat,inplace=True)
test.drop(columns=drop_cols_cat,inplace=True)

## Doing the same for numerical variables 

In [ ]:
num_copy = train.select_dtypes(exclude='object')
num_copy = num_copy.drop(['risk_flag'], axis=1)

In [ ]:
for col in num_copy.columns:
    for col1 in num_copy.columns:
        if col != col1:
            train[col + '_add_' + col1] = train[col] + train[col1]
            train[col + '_minus_' + col1] = train[col] - train[col1]
            test[col + '_add_' + col1] = test[col] + test[col1]
            test[col + '_minus_' + col1] = test[col] - test[col1]

In [ ]:
num_cols=['income', 'age', 'experience', 'current_job_years',
       'current_house_years', 'income_add_age',
       'income_minus_age', 'income_add_experience', 'income_minus_experience',
       'income_add_current_job_years', 'income_minus_current_job_years',
       'income_add_current_house_years', 'income_minus_current_house_years',
       'age_add_experience',
       'age_minus_experience', 'age_add_current_job_years',
       'age_minus_current_job_years', 'age_add_current_house_years','age_minus_current_house_years',
       'experience_add_current_job_years','experience_minus_current_job_years',
       'experience_add_current_house_years','experience_minus_current_house_years',  
       'current_job_years_add_current_house_years','current_job_years_minus_current_house_years']

drop_num_cols=['age_add_income','age_minus_income','experience_add_income','experience_minus_income',
           'current_job_years_add_income','current_job_years_minus_income','current_house_years_add_income',
           'current_house_years_minus_income', 'experience_add_age', 'experience_minus_age',
          'current_job_years_add_age','current_job_years_minus_age','current_house_years_add_age',
          'current_house_years_minus_age','current_job_years_add_experience',
          'current_job_years_minus_experience','current_house_years_add_experience',
          'current_house_years_minus_experience','current_house_years_add_current_job_years',
          'current_house_years_minus_current_job_years']

In [ ]:
train.drop(columns=drop_num_cols,inplace=True)
test.drop(columns=drop_num_cols,inplace=True)

In [ ]:
train.columns[train.dtypes=='object']

In [ ]:
cat_cols=['married', 'house_ownership', 'car_ownership',
       'married_+_house_ownership', 'married_+_car_ownership',
       'married_+_profession', 'married_+_city', 'married_+_state',
       'house_ownership_+_car_ownership', 'house_ownership_+_profession',
       'house_ownership_+_city', 'house_ownership_+_state',
       'car_ownership_+_profession', 'car_ownership_+_city',
       'car_ownership_+_state', 'profession_+_city', 'profession_+_state',
       'city_+_state']

# loo = LeaveOneOutEncoder()
# train[cat_cols] = loo.fit_transform(train[cat_cols],train['risk_flag'])
# test[cat_cols] = loo.transform(test[cat_cols])

In [ ]:
len(train)

In [ ]:
# train.drop_duplicates(inplace=True)

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
cols_to_drop = ['married','car_ownership','house_ownership','married_+_car_ownership',
                'married_+_house_ownership','house_ownership_+_car_ownership','married_+_state',
               'car_ownership_+_state','married_+_profession']

In [ ]:
# train.drop(columns=cols_to_drop,inplace=True)
# test.drop(columns=cols_to_drop,inplace=True)

In [ ]:
# next_drop=['married_+_profession']

# train.drop(columns=next_drop,inplace=True)
# test.drop(columns=next_drop,inplace=True)

In [ ]:
# X=train.drop(columns=['risk_flag'])
# vif_data = pd.DataFrame()
# vif_data["feature"] = X.columns
  
# # calculating VIF for each feature
# vif_data["VIF"] = [variance_inflation_factor(X.values, i)
#                           for i in range(len(X.columns))]
  
# print(vif_data)

In [ ]:
train.columns[train.dtypes!='object']

## Implementing Standard Scaler on numerical columns so that they are all within the same range

In [ ]:
nums = ['income', 'age', 'experience', 'current_job_years',
       'current_house_years', 'income_add_age',
       'income_minus_age', 'income_add_experience', 'income_minus_experience',
       'income_add_current_job_years', 'income_minus_current_job_years',
       'income_add_current_house_years', 'income_minus_current_house_years',
       'age_add_experience', 'age_minus_experience',
       'age_add_current_job_years', 'age_minus_current_job_years',
       'age_add_current_house_years', 'age_minus_current_house_years',
       'experience_add_current_job_years',
       'experience_minus_current_job_years',
       'experience_add_current_house_years',
       'experience_minus_current_house_years',
       'current_job_years_add_current_house_years',
       'current_job_years_minus_current_house_years']

sc = StandardScaler()
train[nums] = sc.fit_transform(train[nums])
test[nums] = sc.transform(test[nums])

In [ ]:
# correlated_features = set()
# correlation_matrix = train.drop('risk_flag', axis=1).corr()

# for i in range(len(correlation_matrix.columns)):
#     for j in range(i):
#         if abs(correlation_matrix.iloc[i, j]) > 0.8:
#             colname = correlation_matrix.columns[i]
#             correlated_features.add(colname)

In [ ]:
# correlated_features

In [ ]:
# train.drop(columns=correlated_features,inplace=True)
# test.drop(columns=correlated_features,inplace=True)

In [ ]:
train.shape

In [ ]:
train.columns

In [ ]:
features=['income', 'age', 'experience', 'married', 'house_ownership',
       'car_ownership', 'current_job_years', 'current_house_years',
       'married_+_house_ownership', 'married_+_car_ownership',
       'married_+_profession', 'married_+_city', 'married_+_state',
       'house_ownership_+_car_ownership', 'house_ownership_+_profession',
       'house_ownership_+_city', 'house_ownership_+_state',
       'car_ownership_+_profession', 'car_ownership_+_city',
       'car_ownership_+_state', 'profession_+_city', 'profession_+_state',
       'city_+_state', 'income_add_age', 'income_minus_age',
       'income_add_experience', 'income_minus_experience',
       'income_add_current_job_years', 'income_minus_current_job_years',
       'income_add_current_house_years', 'income_minus_current_house_years',
       'age_add_experience', 'age_minus_experience',
       'age_add_current_job_years', 'age_minus_current_job_years',
       'age_add_current_house_years', 'age_minus_current_house_years',
       'experience_add_current_job_years',
       'experience_minus_current_job_years',
       'experience_add_current_house_years',
       'experience_minus_current_house_years',
       'current_job_years_add_current_house_years',
       'current_job_years_minus_current_house_years']

In [ ]:
# #import libraries
# X=train.drop(columns=['risk_flag'])
# # from sklearn.svm import SVC
# from sklearn.model_selection import StratifiedKFold
# from sklearn.feature_selection import RFECV
# import matplotlib.pyplot as plt
# #Fit the model
# # svc = SVC(kernel="linear")
# lgb = XGBClassifier(eval_metric='auc')
# rfecv = RFECV(estimator=lgb, step=1,min_features_to_select=20, cv=StratifiedKFold(5), scoring='roc_auc')
# rfecv.fit(X, train['risk_flag'])
# #Selected features
# print(X.columns[rfecv.get_support()])
# print("Optimal number of features : %d" % rfecv.n_features_)

In [ ]:
# X=train.drop(columns=['risk_flag'])
# vif_data = pd.DataFrame()
# vif_data["feature"] = X.columns
  
# # calculating VIF for each feature
# vif_data["VIF"] = [variance_inflation_factor(X.values, i)
#                           for i in range(len(X.columns))]
  
# print(vif_data)

In [ ]:
# features=['married', 'house_ownership', 'car_ownership', 'current_job_years',
#        'current_house_years', 'married_+_house_ownership',
#        'married_+_car_ownership', 'married_+_profession', 'married_+_city',
#        'married_+_state', 'house_ownership_+_car_ownership',
#        'house_ownership_+_state', 'car_ownership_+_profession',
#        'car_ownership_+_state', 'income_minus_experience',
#        'income_add_current_job_years', 'income_minus_current_job_years',
#        'income_add_current_house_years', 'income_minus_current_house_years',
#        'age_add_experience']

In [ ]:
# cat_features=['married', 'house_ownership', 'car_ownership','married_+_house_ownership',
#        'married_+_car_ownership', 'married_+_profession', 'married_+_city',
#        'married_+_state', 'house_ownership_+_car_ownership',
#        'house_ownership_+_state', 'car_ownership_+_profession',
#        'car_ownership_+_state']

In [ ]:
X=train.drop(columns=['risk_flag'])

In [ ]:
# l=[]
# for i in X.columns:
#     if i not in features:
#         l.append(i)

In [ ]:
# X.drop(columns=l,inplace=True)

In [ ]:
# test.drop(columns=l,inplace=True)

In [ ]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [ ]:
data=X
target=train['risk_flag']

## This was the final model which helped me secure a score of 0.74166 on the public leader-board 

In [ ]:
# preds=np.zeros(test.shape[0])
# oof_predictions=np.zeros(len(data))
# kf=StratifiedKFold(n_splits=10,random_state=42,shuffle=True)
# roc=[]
# n=0
# for trn_idx,val_idx in kf.split(data,target):
#     train_x = data.iloc[trn_idx]
#     train_y = target.iloc[trn_idx]
#     val_x = data.iloc[val_idx]
#     val_y = target.iloc[val_idx]
    
#     cb_model = CatBoostClassifier(iterations=5000, learning_rate=0.001,
#                                   custom_metric=['AUC:hints=skip_train~false'], verbose=1000,
# #                                   task_type='GPU'
#                                  )

#     cb_model.fit(train_x, train_y,eval_set=[(val_x,val_y)],
#                      cat_features=categorical_features_indices)
    
# #     lgb_model = LGBMClassifier(
# #         random_state=42,
# #         cat_l2=25.999876242730252,
# #         cat_smooth=89.2699690675538,
# #         colsample_bytree=0.37861069156854954,
# #         early_stopping_round=200,
# #         learning_rate=0.015559547182527743,
# #         max_bin=788,
# #         max_depth=16,
# #         metric="auc",
# #         min_data_per_group=177,
# #         n_estimators=9482,
# #         n_jobs=-1,
# #         num_leaves=244,
# #         reg_alpha=1.8401832133621817e-05,
# #         reg_lambda=0.11900311978519747,
# #         subsample=0.9753842843857927,
# #         subsample_freq=1,
# #         verbose=-1,
# #     )
# #     lgb_model.fit(train_x,train_y,eval_set=[(val_x,val_y)],early_stopping_rounds=100,verbose=False)
#     preds+=cb_model.predict_proba(test[features])[:,1]/kf.n_splits
#     oof_predictions += cb_model.predict_proba(data[features])[:,1]/kf.n_splits
#     roc.append(roc_auc_score( val_y ,cb_model.predict_proba(val_x)[:,1]))
#     print(f"ROC {n+1}: {roc[n]}")
#     n=n+1

In [ ]:
# pred = cb_model.predict_proba(test)

In [ ]:
# cb_model.get_best_score()

In [ ]:
# sub['risk_flag']=[1 if x>0.5 else 0 for x in pred[:,1]]
# sub.to_csv('cb_45.csv',index=False)

In [ ]:
# sum(sub['risk_flag'])

In [ ]:
# feature_imp = pd.DataFrame(sorted(zip(cb_model.feature_importances_,X.columns)), columns=['Value','Feature'])

# plt.figure(figsize=(20, 20))
# sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
# plt.title('CatBoost Features (avg over folds)')
# plt.tight_layout()
# plt.show()
# plt.savefig('lgbm_importances-01.png')